In [1]:
# import packages
import time
from datetime import timedelta, datetime, date

from os.path import join
import os
from os import listdir
from os.path import isfile, join
import csv

import numpy as np
import numpy.matlib
import pandas as pd

import io
from runeq import Config, stream

'''
Download runeq, via Terminal: pip install runeq 
Rune instructions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/00_python_installation_instructions.md
documentation Rune functions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb
'''


'\nDownload runeq, via Terminal: pip install runeq \nRune instructions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/00_python_installation_instructions.md\ndocumentation Rune functions: https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb\n'

In [2]:
path="/Users/roee/Starr_Lab_Folder/Data_Analysis/apple_watch_data/data/download"
os.chdir(path)

In [3]:
# set up config for Rune watch data extraction

'''
Config files are patient and user specific. Open yaml-files to enter personal client key and access key.
To get keys: log in in Rune-labs as admin; click edit-pencil next to patients name;
go to Clients; make new Client; copy keys to yaml-file
'''

def get_yaml_path_dir():
    yaml_path = '/Users/roee/Starr_Lab_Folder/Data_Analysis/apple_watch_data/data/'
    return yaml_path

yaml_path = get_yaml_path_dir()
cfg_RCS02 = Config(os.path.join(yaml_path,'rune_Jeroen_watch_config_RCS02.yaml'))
cfg_RCS02 = Config(os.path.join(yaml_path,'rune_Jeroen_watch_config_RCS05.yaml'))
cfg_RCS02 = Config(os.path.join(yaml_path,'rune_Jeroen_watch_config_RCS06.yaml'))
cfg_RCS02 = Config(os.path.join(yaml_path,'rune_Jeroen_watch_config_RCS07.yaml'))


clients = {'RCS02' : stream.V1Client(cfg_RCS02), 
           'RCS05' : stream.V1Client(cfg_RCS05),
          'RCS06': stream.V1Client(cfg_RCS06),
          'RCS07': stream.V1Client(cfg_RCS07)}






NameError: name 'cfg_RCS05' is not defined

In [ ]:
## RUNE FUNCTIONS (https://github.com/rune-labs/opensource/blob/master/jupyter-notebook-templates/02_download_watch_data.ipynb)
# https://docs.runelabs.io/#tag/v1accel/paths/~1v1~1accel.json/get
'''
in get_accel: expression = "user", to get accelerometry data filtered for gravity direct from Apple-watch
'''
scoreResolution = 60 # enter desired resolution of tremor and dyskinesia scores in seconds

def get_data_download_dir():
    download_path = "/Users/roee/Starr_Lab_Folder/Data_Analysis/apple_watch_data/data/download"
    return download_path
    

def make_full_df(accessor):
    '''Loop through pages of API calls and append to a single dataframe'''
    
    df = pd.DataFrame()
    for page in accessor.iter_csv_text():
        page_df = pd.read_csv(io.StringIO(page))
        df = df.append(page_df, ignore_index=True)

    return df

def get_accel(client, params, save_filepath):
    '''Makes API calls for accel data, saves to CSV and outputs dataframe
    expression=user takes data without gravity from watch'''

    accessor = client.Accel(**params, expression= "user" )
    df = make_full_df(accessor)
    
    # save dataframe to a csv
    df.to_csv(save_filepath, index=False)
    
    return df

def get_rotation(client, params, save_filepath):
    '''Makes API calls for rotation data, saves to CSV and outputs dataframe'''

    accessor = client.Rotation(**params)
    df = make_full_df(accessor)
    
    # save dataframe to a csv
    df.to_csv(save_filepath, index=False)
    
    return df

def get_tremor(client, params, save_filepath):
    '''Makes API calls for tremor data, saves to CSV and outputs dataframe'''

    accessor = client.ProbabilitySymptom(symptom='tremor', severity='*', **params, resolution=scoreResolution) # resolution is time interval in sec between points
    df = make_full_df(accessor)
    
    # save dataframe to a csv
    df.to_csv(save_filepath, index=False)
    
    return df

def get_dyskinesia(client, params, save_filepath):
    '''Makes API calls for dyskinesia data, saves to CSV and outputs dataframe
    resolution sets number of seconds between two returned points'''

    accessor = client.ProbabilitySymptom(symptom='dyskinesia', **params, resolution=scoreResolution) # resolution is time interval in sec between points
    df = make_full_df(accessor)
    
    # save dataframe to a csv
    df.to_csv(save_filepath, index=False)
    
    return df

In [ ]:
# define download parameters per patient
# device_ids are ID's for patient's Applewatches

# take all days in one download with broad start and end times
d = datetime(2020,5,1) # year,month,day
start = time.mktime(d.timetuple()) #1577865600, is equivalent to: 01/01/2020 @ 12:00am (UTC)
# use current time as end time
# d = datetime.now()
d = datetime(2020,5,20) # year,month,day
end = time.mktime(d.timetuple()) # convert to unix time


downloadParameters = {}

# RCS02
downloadParameters['RCS02'] = {'device_id': 'S9Nj2M1Q', 'start_time': start,
                                 'end_time': end, 'timestamp': 'unix'}
# RCS05
downloadParameters['RCS05'] = {'device_id': 'fkFVvXvO', 'start_time': start,
                                 'end_time': end, 'timestamp': 'unix'}
# RCS06
downloadParameters['RCS06'] = { 'device_id': 'CWTFFG3f',  'start_time': start,
                                 'end_time': end, 'timestamp': 'unix'}
# RCS07
downloadParameters['RCS07'] = {'device_id': '0Z48CCdv', 'start_time': start,
                                 'end_time': end, 'timestamp': 'unix'}



In [ ]:
## Call function to download and save csv's for accelerometry, tremor and dyskinesia scores, per patient
'''
Current saving is one folder, files named per patient and per data-type.
'''
data_dir = get_data_download_dir();
for pt in ['RCS02','RCS05','RCS06','RCS07']:
    get_tremor(clients[pt], downloadParameters[pt], os.path.join(data_dir,'%s_tremor.csv' % pt))
    get_dyskinesia(clients[pt], downloadParameters[pt], os.path.join(data_dir,'%s_dyskinesia.csv' % pt))

#     get_accel(clients[pt], downloadParameters[pt], 
#                       '/Users/jeroenhabets/Desktop/SF data/test download/%s_useraccel.csv' % pt)




In [ ]:
def loadTremorDyskinesiaScores(pt):
    """
    Input: pt : pt name as string
    
    dyskinesia comes with columns: [time, probability]
    tremor comes with columns: ['time', 'mild', 'moderate', 'none', 'slight', 'strong', 'unknown']
    
    returns dictionary of combined scores for patient
    """
        
    # DYSKINESIA PART
    # read csv file
    dyskinesiaFile = pd.read_csv(path+'%s_dyskinesia.csv' % pt , header=0).dropna().reset_index(drop=True) 

    timeStamps = []
    # take for every row the datetime and transofrm it to pd.Timestamp
    for row in np.arange(dyskinesiaFile.shape[0]):
        timeStamps.append(pd.Timestamp(datetime.fromtimestamp(dyskinesiaFile['time'][row])))
    # insert list with pd.Timestamp's as first column
    dyskinesiaFile.insert(loc=0, column='timeStamp', value=timeStamps)
    # delete original column with float-timestamps
    del(dyskinesiaFile['time'])
    dyskinesiaFile = dyskinesiaFile.rename(columns={'probability': 'dyskinesia'})

     ### TREMOR PART
    # read csv file
    tremorFile = pd.read_csv(path+'%s_tremor.csv' % pt , header=0).dropna().reset_index(drop=True) 

    timeStamps = []
    # take for every row the datetime and transofrm it to pd.Timestamp
    for row in np.arange(tremorFile.shape[0]):
        timeStamps.append(pd.Timestamp(datetime.fromtimestamp(tremorFile['time'][row])))
    # insert list with pd.Timestamp's as first column
    tremorFile.insert(loc=0, column='timeStamp', value=timeStamps)
    # delete original column with float-timestamps
    del(tremorFile['time'])
    for col in tremorFile.keys()[1:]:
        tremorFile = tremorFile.rename(columns={col: 'tremor_'+col})



#     ### merging synchronised on timestamps
    data_dir = get_data_download_dir()
    mergedFile = pd.merge_asof(tremorFile, dyskinesiaFile, on='timeStamp').reset_index(drop=True)
    mergedFile.to_csv( os.path.join(data_dir,'%s_totalWatchScores.csv' % pt), index=False)

    return mergedFile
        
        
        
        

In [ ]:
# Call function to load and merge tremor and dyskinesia scores per patient 

watchScores = {} # create one dict which stores dataframes with scores per patient
for pt in ['RCS02','RCS05','RCS06','RCS07']:
    watchScores[pt] = loadTremorDyskinesiaScores(pt)
    